In [1]:
# 1 - 使用efinance获取基金和股票数据，存入日期文件
# 1 - 通过证券宝www.baostock.com获取沪深300指数数据

import re
import pandas as pd
import xlwings as xw
import openpyxl as op
import efinance as ef
import baostock as bs
from datetime import datetime
from datetime import timedelta
from openpyxl import load_workbook

# 使用efinance获取基金和股票数据，存入日期文件

# 基金代码列表
fund_codes = ['510310','162412','110031','006327','000961','012348','501029','003318']
fund_names = ['易方达沪深300','华宝中证医疗', '易方达恒生国企','易方达中概互联','天弘沪深300','天弘恒生科技','华宝红利基金','景顺中证500低波动']

# 股票代码列表
stock_codes = ['春秋航空', '洋河股份', '分众传媒', '腾讯控股', '小米集团', '福寿园']

# 取当前日期转换为20230822格式，适用于efinance
end = datetime.today().strftime("%Y%m%d")
beg = (datetime.today()-timedelta(days=45)).strftime("%Y%m%d")  # 开始日期=当前日期减去15天

# 取当前日期转换为20230822格式，适用于baostock
end_b = datetime.today().strftime("%Y-%m-%d")
beg_b = (datetime.today()-timedelta(days=45)).strftime("%Y-%m-%d")  # 开始日期=当前日期减去15天

file = r'stock'+'\\'+end+'.xlsx'
writer = pd.ExcelWriter(file)
count_fund = 0  # 基金编号计数器
count_stock = 0  # 股票编号计数器

# 获取多只基金数据，存储为xlsx中的多个sheet
for fund_code in fund_codes:
    df_fund = ef.fund.get_quote_history(fund_code)
    df_fund.to_excel(writer, sheet_name=f'{count_fund} - {fund_names[count_fund]}', index=False)
    print(f'基金: {count_fund} - {fund_names[count_fund]} 的行情数据已存储到文件中')
    count_fund = count_fund + 1
print('全部基金数据获取完毕！\n')

# 获取多只股票数据，存储为xlsx中的多个sheet
for stock_code in stock_codes:
    df_stock = ef.stock.get_quote_history(stock_code, beg=beg, end=end)
    df_stock.sort_index(ascending=False,inplace=True) # 按时间倒序
    df_stock.to_excel(writer, sheet_name=f'{count_stock + 8} - {stock_code}', index=False)
    print(f'股票: {count_stock + 8} - {stock_code} 的行情数据已存储到文件中')
    count_stock = count_stock + 1
print('全部股票数据获取完毕！\n')

# 通过证券宝www.baostock.com获取沪深300指数数据
lg = bs.login() # 登陆系统

df_sh300 = bs.query_history_k_data_plus(
    'sh.000300',
    'date,close',
    start_date = beg_b,
    end_date = end_b,
    frequency='d')

# 打印结果集
data_list = []
while (df_sh300.error_code == '0') & df_sh300.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(df_sh300.get_row_data())
df_sh300_result = pd.DataFrame(data_list, columns=df_sh300.fields)
df_sh300_result.sort_index(ascending=False,inplace=True) # 按时间倒序
df_sh300_result.to_excel(writer, sheet_name='14 - 沪深300指数', index=False)
print(f'指数: 14 - 沪深300指数 的行情数据已存储到文件中')
print('全部指数数据获取完毕！')

bs.logout() # 登出系统

writer.close()

基金: 0 - 易方达沪深300 的行情数据已存储到文件中
基金: 1 - 华宝中证医疗 的行情数据已存储到文件中
基金: 2 - 易方达恒生国企 的行情数据已存储到文件中
基金: 3 - 易方达中概互联 的行情数据已存储到文件中
基金: 4 - 天弘沪深300 的行情数据已存储到文件中
基金: 5 - 天弘恒生科技 的行情数据已存储到文件中
基金: 6 - 华宝红利基金 的行情数据已存储到文件中
基金: 7 - 景顺中证500低波动 的行情数据已存储到文件中
全部基金数据获取完毕！

股票: 8 - 春秋航空 的行情数据已存储到文件中
股票: 9 - 洋河股份 的行情数据已存储到文件中
股票: 10 - 分众传媒 的行情数据已存储到文件中
股票: 11 - 腾讯控股 的行情数据已存储到文件中
股票: 12 - 小米集团 的行情数据已存储到文件中
股票: 13 - 福寿园 的行情数据已存储到文件中
全部股票数据获取完毕！

login success!
指数: 14 - 沪深300指数 的行情数据已存储到文件中
全部指数数据获取完毕！
logout success!


In [2]:
# 2-1 添加汇总工作表，并生成周五日期列表

wb = load_workbook(file)

if '汇总' in wb.sheetnames:
    print('汇总表已存在!')
else:
    ws = wb.create_sheet(index=0, title='汇总') # 添加汇总工作簿

print('内容列表：')
for sheet in wb:    
    print('  ', sheet.title)

wb.save(file)
print('\n2-1 - 创建汇总表完毕！')

内容列表：
   汇总
   0 - 易方达沪深300
   1 - 华宝中证医疗
   2 - 易方达恒生国企
   3 - 易方达中概互联
   4 - 天弘沪深300
   5 - 天弘恒生科技
   6 - 华宝红利基金
   7 - 景顺中证500低波动
   8 - 春秋航空
   9 - 洋河股份
   10 - 分众传媒
   11 - 腾讯控股
   12 - 小米集团
   13 - 福寿园
   14 - 沪深300指数

2-1 - 创建汇总表完毕！


In [3]:
# 2-2 创建周五的日期列

# 复制单元格数据
def copy2cell(file_name, wb, ws_src, ws_dsc, src, dsc):
    ws_dsc[dsc].value = src  # 复制单元格的值
    wb.save(file_name)

# 判断给定日期是周几，返回0-6
def isweek(date):
    date_f = datetime.strptime(date, '%Y-%m-%d') # 字符串转为日期格式
    week = datetime.weekday(date_f)
    return(week)

# 创建周五日期列表
def create_fridy(file_name, src_sheet, dsc_sheet, dsc_list, beg, end):
    wb = load_workbook(file_name)
    ws_src = wb[src_sheet]
    ws_dsc = wb[dsc_sheet]
    date_list = [] 
    beg = datetime.strptime(beg, "%Y%m%d") 
    end = datetime.strptime(end, "%Y%m%d")
    
    ws_dsc['A1'].value = '日期'
    ws_dsc['B1'].value = '华宝中证医疗'
    ws_dsc['C1'].value = '易方达恒生国企'
    ws_dsc['D1'].value = '易方达中概互联'
    ws_dsc['E1'].value = '天弘沪深300'
    ws_dsc['F1'].value = '天弘恒生科技'
    ws_dsc['G1'].value = '华宝红利基金'
    ws_dsc['H1'].value = '景顺中证500低波动'
    ws_dsc['I1'].value = '春秋航空'
    ws_dsc['J1'].value = '洋河股份'
    ws_dsc['K1'].value = '分众传媒'
    ws_dsc['L1'].value = '腾讯控股'
    ws_dsc['M1'].value = '小米集团'
    ws_dsc['N1'].value = '福寿园'
    ws_dsc['O1'].value = '易方达沪深300'
    ws_dsc['P1'].value = '沪深300指数'
    
    while end >= beg: # 倒序判断、不断-1天，数据格式转变为列表
        date_str = end.strftime("%Y-%m-%d") 
        date_list.append(date_str) 
        end -= timedelta(days=1)
    s = 2
    # print(date_list)
    for i in date_list:
        # i = str(i.replace('-',''))
        if isweek(i) == 4: # 判断日期为周五
            dsc = dsc_list + str(s) # 计算该写入的位置
            # i = int(i)
            copy2cell(file_name, wb, ws_src, ws_dsc, i, dsc) # 把周五的日期，写入汇总表的B列
            print(i)
            s = s+1
    # print(date_list)
    wb.save(file_name)

create_fridy(file, '8 - 春秋航空', '汇总', 'A', beg, end)

print('\n2-2 - 创建周五日期列表完毕！')

2023-12-29
2023-12-22
2023-12-15
2023-12-08
2023-12-01
2023-11-24
2023-11-17

2-2 - 创建周五日期列表完毕！


In [4]:
# 2-3 根据周五日期列表汇总收盘数据

# 应用透视表方式，循环写入列
def move_fund_dates(file_name, src_sheet, dsc_sheet, dsc_list):
    wb = load_workbook(file_name)
    ws_src = wb[src_sheet]
    ws_dsc = wb[dsc_sheet]

    beg_dsc = dsc_list + str('2')
    end_dsc = dsc_list + str(ws_dsc.max_row)
    beg_src = 'A' + str('2')
    end_src = 'A' + str(ws_src.max_row)
    
    for row in ws_dsc[beg_dsc:end_dsc]:
        for cell in row:
            cell.value = "=VLOOKUP(A{}, '{}'!A2:B100, 2, FALSE)".format(cell.row, src_sheet)
    print('基金：', src_sheet, '汇总完毕！')
    wb.save(file_name)  # 保存文件

def move_stock_dates(file_name, src_sheet, dsc_sheet, dsc_list):
    wb = load_workbook(file_name)
    ws_src = wb[src_sheet]
    ws_dsc = wb[dsc_sheet]

    beg_dsc = dsc_list + str('2')
    end_dsc = dsc_list + str(ws_dsc.max_row)
    beg_src = 'C' + str('2')
    end_src = 'C' + str(ws_src.max_row)
    
    for row in ws_dsc[beg_dsc:end_dsc]:
        for cell in row:
            cell.value = "=VLOOKUP(A{}, '{}'!C2:E100, 3, FALSE)".format(cell.row, src_sheet)
    print('股票：', src_sheet, '汇总完毕！')
    wb.save(file_name)  # 保存文件

# 汇总每周五的A股基金净值收据，老办法，有问题、未使用
def move_fund_dates_1(file_name, src_sheet, dsc_sheet, dsc_list):
    wb = load_workbook(file_name)
    ws_src = wb[src_sheet]
    ws_dsc = wb[dsc_sheet]
    n = 3 # 从C列开始拷贝数据
    for row in ws_src.iter_rows():
        row[0].value = str(row[0].value.replace('-','')) # 日期格式转换
        if row[0].value != '日期' and isweek(row[0].value) == 4 and row[0].value > beg:
            dsc = dsc_list + str(n)
            dsc_date = 'B' + str(n)
            # if row[0].value != ws_dsc[dsc_date].value:
            print(row[0].value)
            print(ws_dsc[dsc_date].value)
            copy2cell(file_name, wb, ws_src, ws_dsc, row[1].value, dsc)
            print('A股基金：', src_sheet, dsc, row[0].value, row[1].value, dsc_date, ws_dsc[dsc_date].value)
            n = n + 1
        elif src_sheet == '3 - 易方达中概互联' or src_sheet == '5 - 天弘恒生科技':
            if row[0].value != '日期' and isweek(row[0].value) == 3 and row[0].value > beg:
                dsc = dsc_list + str(n)
                dsc_date = 'B' + str(n)
                copy2cell(file_name, wb, ws_src, ws_dsc, row[1].value, dsc)
                print('海外基金：', src_sheet, dsc, row[0].value, row[1].value, dsc_date, ws_dsc[dsc_date].value)
                n = n + 1
    wb.save(file_name)  # 保存文件

# 汇总每周五的股票收盘收据，老办法，有问题、未使用
def move_stock_dates_1(file_name, src_sheet, dsc_sheet, dsc_list):
    wb = load_workbook(file_name)
    ws_src = wb[src_sheet]
    ws_dsc = wb[dsc_sheet]
    n = 3
    for row in ws_src.iter_rows():
        row[2].value = str(row[2].value.replace('-',''))
        if row[2].value != '日期' and isweek(row[2].value) == 4 and row[0].value > beg:
            dsc = dsc_list + str(n)
            dsc_date = 'B' + str(n)
            copy2cell(file_name, wb, ws_src, ws_dsc, row[4].value, dsc)
            print('股票：', src_sheet, dsc, row[2].value, row[4].value, dsc, ws_dsc[dsc_date].value)
            n = n +1

# 汇总基金收盘数据
move_fund_dates(file, '1 - 华宝中证医疗', '汇总', 'B')
move_fund_dates(file, '2 - 易方达恒生国企', '汇总', 'C')
move_fund_dates(file, '3 - 易方达中概互联', '汇总', 'D')
move_fund_dates(file, '4 - 天弘沪深300', '汇总', 'E')
move_fund_dates(file, '5 - 天弘恒生科技', '汇总', 'F')
move_fund_dates(file, '6 - 华宝红利基金', '汇总', 'G')
move_fund_dates(file, '7 - 景顺中证500低波动', '汇总', 'H')
move_fund_dates(file, '0 - 易方达沪深300', '汇总', 'O')
move_fund_dates(file, '14 - 沪深300指数', '汇总', 'P')

# 汇总A股收盘数据
move_stock_dates(file, '8 - 春秋航空', '汇总', 'I')
move_stock_dates(file, '9 - 洋河股份', '汇总', 'J')
move_stock_dates(file, '10 - 分众传媒', '汇总', 'K')

# 汇总H股收盘数据
move_stock_dates(file, '11 - 腾讯控股', '汇总', 'L')
move_stock_dates(file, '12 - 小米集团', '汇总', 'M')
move_stock_dates(file, '13 - 福寿园', '汇总', 'N')

print('\n2-3 - 收盘数据汇总完毕！')

基金： 1 - 华宝中证医疗 汇总完毕！
基金： 2 - 易方达恒生国企 汇总完毕！
基金： 3 - 易方达中概互联 汇总完毕！
基金： 4 - 天弘沪深300 汇总完毕！
基金： 5 - 天弘恒生科技 汇总完毕！
基金： 6 - 华宝红利基金 汇总完毕！
基金： 7 - 景顺中证500低波动 汇总完毕！
基金： 0 - 易方达沪深300 汇总完毕！
基金： 14 - 沪深300指数 汇总完毕！
股票： 8 - 春秋航空 汇总完毕！
股票： 9 - 洋河股份 汇总完毕！
股票： 10 - 分众传媒 汇总完毕！
股票： 11 - 腾讯控股 汇总完毕！
股票： 12 - 小米集团 汇总完毕！
股票： 13 - 福寿园 汇总完毕！

2-3 - 收盘数据汇总完毕！


In [5]:
# 2-4 修改汇总数据表格式

from openpyxl.styles import Font # 单元格字体和文字样式
from openpyxl.styles import Alignment # 单元格对齐方式
from openpyxl.styles import PatternFill # 单元格填充颜色
from openpyxl.styles import Side, Border # 单元格边框样式

wb = load_workbook(file)
ws = wb['汇总']

# 对表格进行数据初始化
dfa = pd.read_excel(file, header=1)
dfa.info()

# dfa.columns = ['00', '01', '02', '03', '04','05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15']  # 重命名列
# dfa.iloc['华宝中证医疗'] = dfa.iloc['华宝中证医疗'].astype(float)  # 修改列格式为数字

# 循环
#for row in ws.iter_rows():
#    for cell in row:
#        if cell.value != '':
#            print(cell.value)

# ws['Q3'].number_format = numbers.FORMAT_NUMBER_00
# ws.cell['C'].number_format = NumberFormat('0.00')
# ws.cell['Q'].number_format = NumberFormat('#,##0')
print('\n2-4-1 - 设置数字格式完毕！')

# 修改列宽
ws.column_dimensions['A'].width = 3
ws.column_dimensions['B'].width = 12
ws.column_dimensions['C'].width = 8
ws.column_dimensions['D'].width = 8
ws.column_dimensions['E'].width = 8
ws.column_dimensions['F'].width = 8
ws.column_dimensions['G'].width = 8
ws.column_dimensions['H'].width = 8
ws.column_dimensions['I'].width = 8
ws.column_dimensions['J'].width = 8
ws.column_dimensions['K'].width = 8
ws.column_dimensions['L'].width = 8
ws.column_dimensions['M'].width = 8
ws.column_dimensions['N'].width = 8
ws.column_dimensions['O'].width = 8
ws.column_dimensions['P'].width = 8
ws.column_dimensions['Q'].width = 12
print('\n2-4-2 - 设置列宽完毕！')

# 修改列数据对齐方式
alight = Alignment(
horizontal = 'right', # 水平对齐方式：center/left,right
vertical = 'center' # 垂直对齐方式：center/top/bottom
)

ws.column_dimensions['Q'].alignment = alight
print('\n2-4-3 - 修改列对齐完毕！')

# 修改列数据格式
# ws['C'].number_format = "0.00"
# print('\n2-4-2 - 修改列数据格式完毕！')

# 遍历每个列并设置最合适的宽度，因为有VLOOKUP公式，此方法不适用
# 每列并计算最长单元格的长度，并将最大长度保存在`max_length`变量中，单元格的长度=字符个数+汉字个数*0.7
# 将每列的宽度设置为`(最大长度 + 2) * 1.2`，其中`2`是用于单元格内边距的常数，`1.2`是一个调整因子，以便适应字体大小和风格的变化
  
# for col in ws.columns:
#     max_length = max([
#         len(str(cell.value)) + 0.7 * len(re.findall(r'([\u4e00-\u9fa5])', str(cell.value)))
#         for cell in col
#     ])
#     ws.column_dimensions[col[0].column_letter].width = (max_length + 2) * 1.2

# 保存工作簿
wb.save(file)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   2023-12-29   6 non-null      object 
 1   Unnamed: 1   0 non-null      float64
 2   Unnamed: 2   0 non-null      float64
 3   Unnamed: 3   0 non-null      float64
 4   Unnamed: 4   0 non-null      float64
 5   Unnamed: 5   0 non-null      float64
 6   Unnamed: 6   0 non-null      float64
 7   Unnamed: 7   0 non-null      float64
 8   Unnamed: 8   0 non-null      float64
 9   Unnamed: 9   0 non-null      float64
 10  Unnamed: 10  0 non-null      float64
 11  Unnamed: 11  0 non-null      float64
 12  Unnamed: 12  0 non-null      float64
 13  Unnamed: 13  0 non-null      float64
 14  Unnamed: 14  0 non-null      float64
 15  Unnamed: 15  0 non-null      float64
dtypes: float64(15), object(1)
memory usage: 896.0+ bytes

2-4-1 - 设置数字格式完毕！

2-4-2 - 设置列宽完毕！

2-4-3 - 修改列对齐完毕！
